# Inferencing SSD ONNX model using ONNX Runtime Server

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

SSD is a state-of-the-art object detection model that enables users to identify individual objects in an image, and place bounding boxes around them.

This tutorial uses the Open Neural Network eXchange (ONNX) machine learning model format to show the simplest way to take a pretrained SSD model and start experimenting with object detection. To run through this notebook, you'll need a few dependencies that you can install with the commands below.

## Dependencies

NumPy for data data manipulation:

1) pip install numpy

PIL and Matplotlib for raw image manipulation and inline display:

2) pip install pillow

3) pip install matplotlib

## Pre-requisites to run the notebook

1) Please download the SSD ONNX model (ssd.onnx) file from [here](https://onnxzoo.blob.core.windows.net/models/opset_10/ssd/ssd.onnx)

For example, you could do `wget https://onnxzoo.blob.core.windows.net/models/opset_10/ssd/ssd.onnx` if you have `wget` installed, or you could download the model directly by visiting the referenced URL. 

2) Please download the ONNXRuntime Server image:

`sudo docker pull mcr.microsoft.com/onnxruntime/server`

3) In the same folder as the downloaded `ssd.onnx` file, please run:

`sudo docker run -it -v $(pwd):$(pwd) -e MODEL_ABSOLUTE_PATH=$(pwd)/ssd.onnx -p 9001:8001 mcr.microsoft.com/onnxruntime/server` on Linux. On Windows, you must use WSL. 

(In case of errors like port already allocated etc., please only change the number 9001 to something else (keeping 8001 as is). Please remember the changed port number as it will be be needed to modify the URL where the HTTP request is actually sent. Instructions will be present in python comments in the appropriate Jupyter cell.) 

## Some additional information

There are 2 proto files (*.proto) and the corresponding compiled source code files (*__pb2.py) accompanying this notebook. These were [compiled using protoc](https://developers.google.com/protocol-buffers/docs/pythontutorial). For more documentaion regarding ONNX Runtime server (and predict.proto), please visit [this](https://github.com/microsoft/onnxruntime/blob/master/docs/) page and view the contents of `ONNX_Runtime_Server_Usage.md`.  


In [1]:
# Import some dependency modules that we are going to need to run the SSD model

import numpy as np
import predict_pb2
import onnx_ml_pb2
import requests
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [2]:
# Load the raw image

input_shape = (1, 3, 1200, 1200)
img = Image.open("assets/blueangels.jpg")
img = img.resize((1200, 1200), Image.BILINEAR)

# Let us see what the input image looks like
img

FileNotFoundError: [Errno 2] No such file or directory: 'assets/blueangels.jpg'

In [ ]:
# Preprocess and normalize the image

img_data = np.array(img)
img_data = np.transpose(img_data, [2, 0, 1])
img_data = np.expand_dims(img_data, 0)
mean_vec = np.array([0.485, 0.456, 0.406])
stddev_vec = np.array([0.229, 0.224, 0.225])
norm_img_data = np.zeros(img_data.shape).astype('float32')
for i in range(img_data.shape[1]):
    norm_img_data[:,i,:,:] = (img_data[:,i,:,:]/255 - mean_vec[i]) / stddev_vec[i]

In [ ]:
# Create request message to be sent to the ORT server

input_tensor = onnx_ml_pb2.TensorProto()
input_tensor.dims.extend(norm_img_data.shape)
input_tensor.data_type = 1
input_tensor.raw_data = norm_img_data.tobytes()

request_message = predict_pb2.PredictRequest()
request_message.inputs["image"].data_type = input_tensor.data_type
request_message.inputs["image"].dims.extend(input_tensor.dims)
request_message.inputs["image"].raw_data = input_tensor.raw_data

content_type_headers = ['application/x-protobuf', 'application/octet-stream', 'application/vnd.google.protobuf']

for h in content_type_headers:
    request_headers = {
        'Content-Type': h,
        'Accept': 'application/x-protobuf'
    }

In [ ]:
# Inference run using ORT server
# Change the number 9001 to the appropriate port number if you had changed it during ORT Server docker instantiation

PORT_NUMBER = 9010 # Change if needed
inference_url = "http://10.161.19.27:" + str(PORT_NUMBER) + "/v1/models/ssd/versions/1:predict"
response = requests.post(inference_url, headers=request_headers, data=request_message.SerializeToString())

In [ ]:
# Parse response message

response_message = predict_pb2.PredictResponse()
response_message.ParseFromString(response.content)

bboxes = np.frombuffer(response_message.outputs['bboxes'].raw_data, dtype=np.float32)
labels = np.frombuffer(response_message.outputs['labels'].raw_data, dtype=np.int64)
scores = np.frombuffer(response_message.outputs['scores'].raw_data, dtype=np.float32)

print('Boxes shape:', response_message.outputs['bboxes'].dims)
print('Labels shape:', response_message.outputs['labels'].dims)
print('Scores shape:', response_message.outputs['scores'].dims)

In [ ]:
## Display image with bounding boxes and appropriate class 

# Parse the list of class labels
classes = [line.rstrip('\n') for line in open('assets/coco_classes.txt')]

# Plot the bounding boxes on the image
plt.figure()
fig, ax = plt.subplots(1, figsize=(12,9))
ax.imshow(img)

resized_width = 1200  # we resized the original image, remember ? 
resized_height = 1200
num_boxes = 6 # we limit displaying to just 10 boxes to avoid clogging the result image with boxes
               # The results are already sorted based on box confidences, so we just pick top N boxes without sorting
    
for c in range(num_boxes):    
    base_index = c * 4
    y1, x1, y2, x2 = bboxes[base_index] * resized_height, bboxes[base_index + 1] * resized_width, bboxes[base_index + 2] * resized_height, bboxes[base_index + 3] * resized_width 
    color = 'blue'
    box_h = (y2 - y1)
    box_w = (x2 - x1)
    bbox = patches.Rectangle((y1, x1), box_h, box_w, linewidth=2, edgecolor=color, facecolor='none')
    ax.add_patch(bbox)
    plt.text(y1, x1, s=classes[labels[c] - 1], color='white', verticalalignment='top', bbox={'color': color, 'pad': 0})
plt.axis('off')

# Save image
plt.savefig("result.jpg", bbox_inches='tight', pad_inches=0.0)
plt.show()
